In [2]:
!pip install pandas numpy scikit-learn requests joblib


In [3]:
import pandas as pd
import numpy as np
import requests
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import datetime

print("All libraries imported successfully!")


All libraries imported successfully!


In [4]:
data = pd.DataFrame({
    'pm25': [45, 60, 80, 90],
    'pm10': [110, 140, 160, 180],
    'temp': [22, 25, 20, 18],
    'humidity': [60, 55, 70, 75],
    'wind': [2.1, 3.0, 1.5, 1.2],
    'hour': [10, 12, 18, 20],
    'month': [1, 2, 11, 12],
    'future_pm25': [52, 68, 95, 120]
})

data


,pm25,pm10,temp,humidity,wind,hour,month,future_pm25
0,45,110,22,60,2.1,10,1,52
1,60,140,25,55,3.0,12,2,68
2,80,160,20,70,1.5,18,11,95
3,90,180,18,75,1.2,20,12,120


In [5]:
X = data[['pm25', 'pm10', 'temp', 'humidity', 'wind', 'hour', 'month']]
y = data['future_pm25']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print("Model MAE:", mean_absolute_error(y_test, predictions))


Model MAE: 2.019999999999996


In [6]:
joblib.dump(model, "air_pollution_model.pkl")
print("Model saved!")


Model saved!


In [13]:
def get_air_quality(city):
    url = f"https://api.openaq.org/v2/latest?city={city}"
    response = requests.get(url).json()

    # Default fallback values (safe & realistic)
    pm25 = 50
    pm10 = 100

    if 'results' not in response or len(response['results']) == 0:
        print("⚠ Air quality data not available, using fallback values")
        return pm25, pm10

    for m in response['results'][0]['measurements']:
        if m['parameter'] == 'pm25':
            pm25 = m['value']
        if m['parameter'] == 'pm10':
            pm10 = m['value']

    return pm25, pm10



In [16]:
def get_weather(city):
    url = (
        f"https://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={API_KEY}&units=metric"
    )

    response = requests.get(url)
    data = response.json()

    if response.status_code != 200:
        print("❌ Weather API error:", data)
        return None, None, None

    return (
        data['main']['temp'],
        data['main']['humidity'],
        data['wind']['speed']
    )



In [22]:
city = "Mumbai"

pm25, pm10 = get_air_quality(city)
temp, humidity, wind = get_weather(city)

if temp is None:
    print("❌ Weather data unavailable. Try again later.")
else:
    now = datetime.datetime.now()

    input_data = pd.DataFrame([[
        pm25, pm10, temp, humidity, wind, now.hour, now.month
    ]], columns=[
        'pm25','pm10','temp','humidity','wind','hour','month'
    ])

    prediction = model.predict(input_data)[0]

    print("Current PM2.5:", pm25)
    print("Predicted PM2.5 (Next Day):", round(prediction, 2))


⚠ Air quality data not available, using fallback values
Current PM2.5: 50
Predicted PM2.5 (Next Day): 73.56


In [23]:
def aqi_category(pm25):
    if pm25 <= 30:
        return "Good 🟢"
    elif pm25 <= 60:
        return "Satisfactory 🟡"
    elif pm25 <= 90:
        return "Moderate 🟠"
    elif pm25 <= 120:
        return "Poor 🔴"
    elif pm25 <= 250:
        return "Very Poor 🟣"
    else:
        return "Severe ⚫"


In [24]:
category = aqi_category(prediction)

print("📍 City:", city)
print("🌡 Temperature:", temp, "°C")
print("💨 Wind Speed:", wind, "m/s")
print("🫁 Current PM2.5:", pm25)
print("📈 Predicted PM2.5 (Next Day):", round(prediction, 2))
print("🏷 Air Quality Category:", category)


📍 City: Mumbai
🌡 Temperature: 27.99 °C
💨 Wind Speed: 2.57 m/s
🫁 Current PM2.5: 50
📈 Predicted PM2.5 (Next Day): 73.56
🏷 Air Quality Category: Moderate 🟠
